In [1]:
%pip install datasets


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 468.7/468.7 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 14.8/14.8 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 20.5/20.5 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 11.2/11.2 MB 2.9 MB/s eta 0:00:00
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     -------------------------------------- 134.3/134.3 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 154.0/154.0 kB 9.6 MB/s eta 0:00:00
     -------------------------------------- 317.2/317.2 kB 6.5 MB/s eta 0:00:00
     -------------------------------------- 200.1/200.1 kB 2.0 MB/s eta 0:00:00
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     -------------------------------------- 143.2/143.2 kB 4.2 MB/s eta 0:00:00
  Using cached attrs-22.2.0-py3-no

In [1]:
#importing datasets
from datasets import load_dataset

c:\Users\natal\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#loading train dataset
dataset_train = load_dataset("GonzaloA/fake_news", split="train")

Found cached dataset parquet (C:/Users/natal/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-1fe2b42e1fa111c8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
#loading test dataset
dataset_test = load_dataset("GonzaloA/fake_news", split="test")

Found cached dataset parquet (C:/Users/natal/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-1fe2b42e1fa111c8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
#loading validation dataset 
dataset_validation = load_dataset("GonzaloA/fake_news", split="validation")

Found cached dataset parquet (C:/Users/natal/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-1fe2b42e1fa111c8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [3]:
import pandas as pd 

In [4]:
#converting datasets into dataframes
df_train = pd.DataFrame(dataset_train)

In [8]:
#converting datasets into dataframes
df_test = pd.DataFrame(dataset_test)

In [9]:
#converting datasets into dataframes
df_validate = pd.DataFrame(dataset_validation)

In [5]:
#dropping unnamed column
df_train.drop('Unnamed: 0', inplace=True, axis=1)

In [11]:
#dropping unnamed column
df_test.drop('Unnamed: 0', inplace=True, axis=1)

In [12]:
#dropping unnamed column
df_validate.drop('Unnamed: 0', inplace=True, axis=1)

In [6]:
#renaming columns, dropping NAN values, and dropping the text column
df = df_train.rename(columns={'title':'statement'})
df = df.dropna(subset=['statement']).reset_index(drop=True)
df = df.drop(['text'], axis=1)

In [7]:
import re

Sentence Negation Pre-processing

In [8]:
#methods for negating sentences
negate_dict = {" isn't ":" is ",
    " isn\'t ":" is ",
    " is not ":" is ",
    " is ":" is not ",
    " didn't ":" did ",
    " didn\'t ":" did ",
    " did not ":" did ",
    " does not have ":" has ",
    " doesn't have ":" has ",
    " doesn\'t have ":" has ",
    " has ":" does not have ",
    " shouldn't ":" should ",
    " shouldn\'t ":" should ",
    " should not ":" should ",
    " should ":" should not ",
    " wouldn't ":" would ",
    " wouldn\'t ":" would ",
    " would not ":" would ",
    " would ":" would not ",
    " mustn't ":" must ",
    " mustn\'t ":" must ",
    " must not ":" must ",
    " must ":" must not ",
    " can't ":" can ",
    " can\'t ":" can ",
    " cannot ":" can ",
    " can ":" cannot "} 

IRREGULAR_ES_VERB_ENDINGS = ["ss", "x", "ch", "sh", "o"]

def negate(sentence):

  for key in negate_dict.keys():
    if sentence.find(key) > -1:
      return sentence.replace(key, negate_dict[key])



  if re.search("doesn't", sentence):
    return re.sub("doesn't", "does not", sentence, 1)

  return None

def __is_consonant(letter):
  return letter not in ['a', 'e', 'i', 'o', 'u', 'y']


def replace_verb(matchobj):
  subject = matchobj.group(1)
  verb = matchobj.group(2)
  whitespace = matchobj.group(3)

  if verb.endswith("y") and __is_consonant(verb[-2]):
        return "{0}ies".format(verb[0:-1])

  # flies -> fly, but not die -> dy
  if verb.endswith("ie") and len(verb) > 3:
    verb = "{0}y".format(verb[0:-2])

  # stresses -> stress
  for ending in IRREGULAR_ES_VERB_ENDINGS:
        
    if verb.endswith(ending):
      return "{0}es".format(verb)    
    elif verb.endswith("{0}e".format(ending)):
      verb = verb[0:-1]

  return "{0}{1}{2}".format(subject, verb, whitespace)

In [9]:
df['statement'] = df['statement'].apply(lambda x: x.lower().replace('’',"'"))
df_neg = df.copy()
df_neg['statement'] = df_neg['statement'].apply(negate)

df_neg = df_neg.loc[~df_neg.statement.isnull()]
df_pos = df.loc[df_neg.index].reset_index(drop=True)
df_neg = df_neg.reset_index(drop=True)

In [11]:
df

,statement,label
0,‘maury' show official facebook posts f*cked u...,0
1,trump's favorite news channel tries to soothe...,0
2,"russia warns iraq, kurds not to destabilize mi...",1
3,watch steve scalise throw a strike at the nati...,0
4,trump will hate what stephen colbert just did...,0
...,...,...
24348,eu parliament chief asks poland to ensure meps...,1
24349,chemical weapons watchdog found sarin used in ...,1
24350,"melissa harris-perry is done with msnbc, pens...",0
24351,trump's pick for navy secretary withdraws,1


In [10]:
df_neg

,statement,label
0,eu's verhofstadt pokes fun at british pm may b...,1
1,most unwanted man in the world: argentinians d...,0
2,donald trump jr. says bragging about sexual a...,0
3,china says war must be allowed on korean penin...,1
4,"in alabama's senate race, contenders fight ove...",1
...,...,...
2980,hillary's secret weapon for the end of her ca...,0
2981,get out of jail free: how obama's race war is ...,0
2982,mike pence is not fine with trump sexual assau...,1
2983,trump unravels as republicans admit he should...,0


In [217]:
df_neg.to_csv('C:/Users/natal/Documents/NLP/negated_data.csv')

Name Reversal Pre-processing

In [12]:
import spacy
from fuzzywuzzy import fuzz, process
import random
import itertools

c:\Users\natal\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB 9.6 MB/s eta 0:00:02
     - -------------------------------------- 0.6/12.8 MB 7.6 MB/s eta 0:00:02
     -- ------------------------------------- 0.7/12.8 MB 5.9 MB/s eta 0:00:03
     --- ------------------------------------ 1.1/12.8 MB 6.1 MB/s eta 0:00:02
     --- ------------------------------------ 1.2/12.8 MB 5.4 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.8 MB 5.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 4.4 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 4.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.6/12.8 MB 4.0 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 4.0 MB/s eta 0:00:03
     ------ --------------------------------- 2.0/12.8 MB 3.9 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 3

In [14]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

In [15]:
def find_entities(text, type='PERSON'):
    doc = nlp(text)
    return [entity.text for entity in doc.ents if (entity.label_==type)\
          and (len(entity.text.split())>1)]

In [16]:
df_entities = df.copy()

In [ ]:
df_entities['entities'] = df_entities['statement'].apply(find_entities)

In [130]:
df_entity = df_entities.copy()

In [131]:
names = list(set(list(itertools.chain(*df_entity['entities']))))
df_pol = pd.DataFrame({'name':names})

In [173]:
def replace_names(text, lst1, lst2):
    for (a,b) in zip(lst1,lst2):
        text = text.replace(a,b)
    return text

In [82]:
from sklearn.utils import shuffle

In [162]:
df_pol_shuffled = shuffle(df_pol)

In [174]:
#generate list of candidates
df_entity['statement_new'] = df_entity.apply(lambda row: replace_names(row['statement'],row['entities'],list(df_pol_shuffled['name'])), axis=1)

In [ ]:
df_entity.drop('statement',inplace=True,axis=1)

In [ ]:
df_entity.drop('entities',inplace=True,axis=1)

In [184]:
df_entity.drop('random entity',inplace=True,axis=1)


In [188]:
df_entity.rename(columns={'statement_new':'statement'},inplace=True)

In [216]:
df_entity

,statement,label
0,‘Maury’ Show Official Facebook Posts F*CKED U...,0
1,Trump’s Favorite News Channel Tries To Soothe...,0
2,"Russia warns Iraq, Kurds not to destabilize Mi...",1
3,WATCH STEVE SCALISE Throw A Strike At The Nati...,0
4,Trump Will HATE What Ruth Bader Just Did To H...,0
...,...,...
24348,EU Parliament chief asks Poland to ensure MEPs...,1
24349,Chemical weapons watchdog found sarin used in ...,1
24350,"Ruth Bader Is DONE With MSNBC, Pens Detailed ...",0
24351,Trump's pick for Navy secretary withdraws,1


In [215]:
columns_titles = ['statement','label']
df_entity=df_entity.reindex(columns=columns_titles)

In [218]:
df_entity.to_csv('C:/Users/natal/Documents/NLP/name_switched_data.csv')

Adverb Intensity Pre-processing

In [204]:
BOOSTER_DICT = ['absolutely', 'amazingly', 'awfully', 'barely',
                'completely', 'considerably', 'decidedly', 'deeply', 
                'enormously', 'entirely', 'especially', 'exceptionally',
                'exclusively', 'extremely', 'fully', 'greatly', 'hardly',
                'hella', 'highly', 'hugely', 'incredibly', 'intensely',
                'majorly', 'overwhelmingly', 'really', 'remarkably',
                'substantially', 'thoroughly', 'totally', 'tremendously',
                'unbelievably', 'unusually', 'utterly', 'very']

In [205]:
df_adverb = df.copy()

In [206]:
df_adverb['polar_word'] = df_adverb['statement'].apply(lambda s: any(w in BOOSTER_DICT for w in s.split())) 
df_adverb = df_adverb.loc[df_adverb.polar_word].reset_index(drop=True)
df_adverb['statement_new'] = df_adverb['statement'].apply(lambda s: ' '.join([w for w in s.split() if w.lower() not in BOOSTER_DICT]))

In [208]:
df_adverb

,statement,label,polar_word,statement_new
0,EU's Mogherini: U.S. says will fully implement...,1,True,EU's Mogherini: U.S. says will implement Iran ...
1,wh press secretary says obamas denial about cl...,1,True,wh press secretary says obamas denial about cl...
2,FBI INVESTIGATION Into HILLARY’S Email Server ...,0,True,FBI INVESTIGATION Into HILLARY’S Email Server ...
3,"MAXINE WATERS: Obama’s Left A “very, very powe...",0,True,"MAXINE WATERS: Obama’s Left A “very, powerful”..."
4,UNBELIEVABLE! VINTAGE VIDEO EXPOSES Racist Fir...,0,True,UNBELIEVABLE! VINTAGE VIDEO EXPOSES Racist Fir...
...,...,...,...,...
56,"Trump says he is 'very, very close' to making ...",1,True,"Trump says he is 'very, close' to making Fed c..."
57,Vice President Pence: Trump greatly concern ab...,1,True,Vice President Pence: Trump concern about Irma...
58,RUSSIA HITS OBAMA…HARD: “We are reaching a rea...,0,True,RUSSIA HITS OBAMA…HARD: “We are reaching a ter...
59,leaked emails reveal hillary cant speak for ve...,0,True,leaked emails reveal hillary cant speak for lo...


In [209]:
df_adverb.drop('polar_word',inplace=True,axis=1)

In [210]:
df_adverb.drop('statement',inplace=True,axis=1)

In [211]:
df_adverb.rename(columns={'statement_new':'statement'},inplace=True)

In [213]:
columns_titles = ['statement','label']
df_adverb=df_adverb.reindex(columns=columns_titles)

In [219]:
df_adverb.to_csv('C:/Users/natal/Documents/NLP/adverb_intensity_data.csv')